In [25]:
class A: pass
class B: pass
class C(B): pass
class D(A, C): pass

In [26]:
D.mro()

[__main__.D, __main__.A, __main__.C, __main__.B, object]

In [31]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
#     @classmethod
#     def instance(cls):
#         ...


class Chair(Singleton):
    def __init__(self, val):
        print(f"init with {val}")
        self.val = val
       
    
c1 = Chair(10)
c2 = Chair(20)


print(c1 is c2, c1.val, c2.val)

init with 10
init with 20
True 20 20


In [37]:
class Connector:
    def __init__(self, db_name):
        self.conn = Chair(db_name)
    
    def __del__(self):
        #self.conn.close()
        print("DEL")

db_conn = Connector("db")
db_conn1 = db_conn

del db_conn
del db_conn1

init with db
DEL


In [39]:
class Attr:
    def __set_name__(self, owner, name):
        print(f"{locals()=}")
        self.name = name
        

class A:
    x = Attr()  # Automatically calls: x.__set_name__(A, "x")

    
a = A()

locals()={'self': <__main__.Attr object at 0x10b0b10f0>, 'owner': <class '__main__.A'>, 'name': 'x'}


In [40]:
a.x.name

'x'

In [56]:
class AttrAccess:
    name = "cls_AttrAccess"

    def __init__(self, val):
        self.val = val

    def __getattribute__(self, name):
        print(f"__getattribute__ {name=}")
        return super().__getattribute__(name)
        
    def __getattr__(self, name):
        print(f"__getattr__ {name=}")
        raise AttributeError("no attr {name}")
#         return "no_attr_but_default"
        
        #return super().__getattr__(name)
        
    def __setattr__(self, name, val):
        print(f"__setattr__ {name=}, {val=}")
        return super().__setattr__(name, val)
    
    def __delattr__(self, name):
        print(f"__delattr__ {name=}")
        return super().__delattr__(name)
    
aa = AttrAccess(42)

__setattr__ name='val', val=42


In [47]:
aa.val

__getattribute__ name='val'


42

In [48]:
aa.new_val = 99

__setattr__ name='new_val', val=99


In [49]:
aa.new_val

__getattribute__ name='new_val'


99

In [50]:
del aa.new_val

__delattr__ name='new_val'


In [57]:
aa.new_name

__getattribute__ name='new_name'
__getattr__ name='new_name'


AttributeError: no attr {name}

In [58]:
aa.name

__getattribute__ name='name'


'cls_AttrAccess'

In [59]:
AttrAccess.name

'cls_AttrAccess'

In [107]:
class IntegerField:
    
    def __init__(self):
        print(f"IntField __init__")
        self._instance_attr_name = "_integer_field"

    def __get__(self, obj, objtype):
        print(f"get {obj} cls={objtype}")
        
        if obj is None:
            return
        
        return getattr(obj, self._instance_attr_name)

    def __set__(self, obj, val):
        print(f"set {val} for {obj}")
        
        if obj is None:
            return
        
        return setattr(obj, self._instance_attr_name, val)

    def __delete__(self, obj):
        print(f"delete from {obj}")
        
        if obj is None:
            return

        return delattr(obj, self._instance_attr_name)


class TableUser:
    age = IntegerField()
    birth_year = IntegerField()
    
    def __init__(self, age, year):
        self.age = age
        self.birth_year = year

IntField __init__
IntField __init__


In [92]:
TableUser.age

get None cls=<class '__main__.TableUser'>


In [94]:
user = TableUser(42)

set 42 for <__main__.TableUser object at 0x10b147c10>


In [95]:
user.age

get <__main__.TableUser object at 0x10b147c10> cls=<class '__main__.TableUser'>


42

In [99]:
user.__dict__

{'_integer_field': 99}

In [97]:
user.age = 99

set 99 for <__main__.TableUser object at 0x10b147c10>


In [98]:
user.age

get <__main__.TableUser object at 0x10b147c10> cls=<class '__main__.TableUser'>


99

In [101]:
user2 = TableUser(34)
user2.age

set 34 for <__main__.TableUser object at 0x10b1476d0>
get <__main__.TableUser object at 0x10b1476d0> cls=<class '__main__.TableUser'>


34

In [102]:
user2.__dict__, user.__dict__, TableUser.__dict__

({'_integer_field': 34},
 {'_integer_field': 99},
 mappingproxy({'__module__': '__main__',
               'age': <__main__.IntegerField at 0x10b146b00>,
               '__init__': <function __main__.TableUser.__init__(self, age)>,
               '__dict__': <attribute '__dict__' of 'TableUser' objects>,
               '__weakref__': <attribute '__weakref__' of 'TableUser' objects>,
               '__doc__': None}))

In [86]:
TableUser.__dict__["age"].__get__

<bound method IntegerField.__get__ of <__main__.IntegerField object at 0x10acb7df0>>

In [103]:
user.age, user2.age

get <__main__.TableUser object at 0x10b147c10> cls=<class '__main__.TableUser'>
get <__main__.TableUser object at 0x10b1476d0> cls=<class '__main__.TableUser'>


(99, 34)

In [108]:
user = TableUser(42, 1896)

set 42 for <__main__.TableUser object at 0x10b1473a0>
set 1896 for <__main__.TableUser object at 0x10b1473a0>


In [111]:
user.age, user.birth_year, user.__dict__

get <__main__.TableUser object at 0x10b1473a0> cls=<class '__main__.TableUser'>
get <__main__.TableUser object at 0x10b1473a0> cls=<class '__main__.TableUser'>


(1896, 1896, {'_integer_field': 1896})

In [123]:
class BadIntegerError(Exception):
    pass


class IntegerField:
    
    def __set_name__(self, owner, name):
        self._instance_attr_name = f"_int_field_{name}"
        print(f"IntField set_name {self._instance_attr_name}")

    def __get__(self, obj, objtype):
        print(f"get {obj} cls={objtype}")
        
        if obj is None:
            return
        
        return getattr(obj, self._instance_attr_name)

    def __set__(self, obj, val):
        print(f"set {val} for {obj}")
        
        if obj is None:
            return
        
        if not isinstance(val, int):
            raise BadIntegerError(f"{val=} is not an integer!")
        
        return setattr(obj, self._instance_attr_name, val)

    def __delete__(self, obj):
        print(f"delete from {obj}")
        
        if obj is None:
            return

        return delattr(obj, self._instance_attr_name)


class TableUser:
    age = IntegerField()
    birth_year = IntegerField()
    
    def __init__(self, age, year):
        self.age = age
        self.birth_year = year

IntField set_name _int_field_age
IntField set_name _int_field_birth_year


In [124]:
user = TableUser(42, 1896)

set 42 for <__main__.TableUser object at 0x10aa39090>
set 1896 for <__main__.TableUser object at 0x10aa39090>


In [114]:
user.age, user.birth_year, user.__dict__

get <__main__.TableUser object at 0x10ae82350> cls=<class '__main__.TableUser'>
get <__main__.TableUser object at 0x10ae82350> cls=<class '__main__.TableUser'>


(42, 1896, {'_int_field_age': 42, '_int_field_birth_year': 1896})

In [115]:
del user.age

delete from <__main__.TableUser object at 0x10ae82350>


In [116]:
user.age

get <__main__.TableUser object at 0x10ae82350> cls=<class '__main__.TableUser'>


AttributeError: 'TableUser' object has no attribute '_int_field_age'

In [117]:
user.__dict__

{'_int_field_birth_year': 1896}

In [119]:
user.age = 1

set 1 for <__main__.TableUser object at 0x10ae82350>


In [120]:
user.__dict__

{'_int_field_birth_year': 1896, '_int_field_age': 1}

In [121]:
user.__dict__["age"] = 1000

In [122]:
user.age

get <__main__.TableUser object at 0x10ae82350> cls=<class '__main__.TableUser'>


1

In [126]:
user = TableUser(42, 1896)
user.age = "99"

set 42 for <__main__.TableUser object at 0x10aa38640>
set 1896 for <__main__.TableUser object at 0x10aa38640>
set 99 for <__main__.TableUser object at 0x10aa38640>


BadIntegerError: val='99' is not an integer!

In [145]:
class AMeta(type):
    def __new__(mcs, name, bases, classdict, **kwargs):
        cls = super().__new__(mcs, name, bases, classdict)
        print('Meta __new__', cls)

        return cls

    def __init__(cls, name, bases, classdict, **kwargs):
        print('Meta __init__', cls, name, bases, classdict)
        super().__init__(name, bases, classdict, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('Meta __call__', cls, args, kwargs)
        return super().__call__(*args, **kwargs)

    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('Meta __prepare__', mcs, name, bases, kwargs)
        
        return {'added_attr_a': 2, 'added_attr_b': 99}

In [159]:
class TableChair(metaclass=AMeta):
    name = "some_name"
    
    def __new__(cls, *args, **kwargs):
        print(f"TableChair.__new__ {cls}", args, kwargs)
        
        return super().__new__(cls)
    
    def __init__(self, val):
        self.val = val
        print(f"TableChair.__init__ {val=}")
    

Meta __prepare__ <class '__main__.AMeta'> TableChair () {}
Meta __new__ <class '__main__.TableChair'>
Meta __init__ <class '__main__.TableChair'> TableChair () {'added_attr_a': 2, 'added_attr_b': 99, '__module__': '__main__', '__qualname__': 'TableChair', 'name': 'some_name', '__new__': <function TableChair.__new__ at 0x10aa25000>, '__init__': <function TableChair.__init__ at 0x10aa268c0>, '__classcell__': <cell at 0x10a9a8310: AMeta object at 0x7fd99402a980>}


In [160]:
chair = TableChair(99)

Meta __call__ <class '__main__.TableChair'> (99,) {}
TableChair.__new__ <class '__main__.TableChair'> (99,) {}
TableChair.__init__ val=99


In [153]:
type(chair)

NoneType

In [147]:
type(TableChair)

__main__.AMeta

In [149]:
isinstance(TableChair, type), isinstance(TableChair, object)

(True, True)

In [150]:
isinstance(type, object), isinstance(object, type)

(True, True)

In [140]:
Boo = AMeta("Boo", (Foo,), {"attr": 99})

Meta __new__ <class '__main__.Boo'>
Meta __init__ <class '__main__.Boo'> Boo (<class '__main__.Foo'>,) {'attr': 99}


In [141]:
type(Boo)

__main__.AMeta

In [143]:
isinstance(Boo, AMeta), isinstance(Boo, type)

(True, True)

In [144]:
Boo.attr

99

In [128]:
Foo = type("Foo", (), {})

In [132]:
x = Foo()
print(x)

In [137]:
type(x), type(Foo), type(type), type(object)

(__main__.Foo, type, type, type)

In [161]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance


class Chair(Singleton):
    def __init__(self, val):
        print(f"init with {val}")
        self.val = val
       
    
c1 = Chair(10)
c2 = Chair(20)


print(c1 is c2, c1.val, c2.val)

init with 10
init with 20
True 20 20


In [166]:
class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        print('Meta __call__', cls, args, kwargs)
        
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)

        return cls._instances[cls]


class Sofa(metaclass=MetaSingleton):
    def __new__(cls, *args, **kwargs):
        print(f"Sofa.__new__ {cls}", args, kwargs)
        
        return super().__new__(cls)

    def __init__(self, val):
        print(f"Sofa.init with {val}")
        self.val = val


print("========")

s1 = Sofa(10)
s2 = Sofa(20)


print(s1 is s2, s1.val, s2.val)

Meta __call__ <class '__main__.Sofa'> (10,) {}
Sofa.__new__ <class '__main__.Sofa'> (10,) {}
Sofa.init with 10
Meta __call__ <class '__main__.Sofa'> (20,) {}
True 10 10


In [167]:
Sofa._instances

{__main__.Sofa: <__main__.Sofa at 0x10ae80280>}

In [168]:
class SoftSofa(Sofa):
    pass

In [169]:
soft1 = SoftSofa(11)
soft2 = SoftSofa(22)

soft1 is soft2

Meta __call__ <class '__main__.SoftSofa'> (11,) {}
Sofa.__new__ <class '__main__.SoftSofa'> (11,) {}
Sofa.init with 11
Meta __call__ <class '__main__.SoftSofa'> (22,) {}


True

In [170]:
SoftSofa._instances

{__main__.Sofa: <__main__.Sofa at 0x10ae80280>,
 __main__.SoftSofa: <__main__.SoftSofa at 0x10ae80250>}

In [171]:
Sofa._instances

{__main__.Sofa: <__main__.Sofa at 0x10ae80280>,
 __main__.SoftSofa: <__main__.SoftSofa at 0x10ae80250>}

In [192]:
class Shape:
    def __init__(self, name):
        self.name = name
        self.points = []
    
    def draw(self):
        print(f"{self.__class__.__name__}.draw")

        
class ColoredShape(Shape):
    def __init__(self, color, *args, **kwargs):
        self.color = color
        super().__init__(*args, **kwargs)
        
    def draw(self):
        print(f"ColoredShape {self.color=}")
        
        super().draw()
        
        
class BoldShape(Shape):
    def __init__(self, width, *args, **kwargs):
        self.width = width
        super().__init__(*args, **kwargs)
        
    def draw(self):
        print(f"BoldShape {self.width=}")
        
        super().draw()

        
class BoldColoredShape(BoldShape, ColoredShape):

    def draw(self):
        print(f"BoldColoredShape {self.__dict__=}")
        
        super().draw()

In [175]:
col = ColoredShape("green", "circle")
col.draw()

ColoredShape.draw


In [180]:
col = ColoredShape("green", "circle")
col.draw()

ColoredShape self.color='green'


In [182]:
col = ColoredShape("green", "circle")
col.draw()

ColoredShape self.color='green'
ColoredShape.draw


In [184]:
bold = BoldShape(14, "circle")
bold.draw()

BoldShape self.width=14
BoldShape.draw


In [188]:
BoldColoredShape.mro()

[__main__.BoldColoredShape,
 __main__.BoldShape,
 __main__.ColoredShape,
 __main__.Shape,
 object]

In [193]:
bc = BoldColoredShape(42, "yellow", "circle")

In [194]:
bc.__dict__

{'width': 42, 'color': 'yellow', 'name': 'circle', 'points': []}

In [195]:
bc.draw()

BoldColoredShape self.__dict__={'width': 42, 'color': 'yellow', 'name': 'circle', 'points': []}
BoldShape self.width=42
ColoredShape self.color='yellow'
BoldColoredShape.draw
